In [ ]:
import numpy as np
import random as rn
import tensorflow as tf
import datetime

np.random.seed(30)
rn.seed(30)
tf.random.set_seed(30)

In [8]:
import os
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GRU, Conv3D, MaxPooling3D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import pandas as pd


In [9]:
base_dir = '/home/datasets/Project_data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

In [13]:
train_csv = pd.read_csv(os.path.join(base_dir, 'train.csv'))
val_csv = pd.read_csv(os.path.join(base_dir, 'val.csv'))

print("Train CSV Sample:")
display(train_csv.head())
print("Validation CSV Sample:")
display(val_csv.head())

Train CSV Sample:


,WIN_20180925_17_08_43_Pro_Left_Swipe_new;Left_Swipe_new;0
0,WIN_20180925_17_18_28_Pro_Left_Swipe_new;Left_...
1,WIN_20180925_17_18_56_Pro_Left_Swipe_new;Left_...
2,WIN_20180925_17_19_51_Pro_Left_Swipe_new;Left_...
3,WIN_20180925_17_20_14_Pro_Left_Swipe_new;Left_...
4,WIN_20180925_17_21_28_Pro_Left_Swipe_new;Left_...


Validation CSV Sample:


,WIN_20180925_17_17_04_Pro_Left_Swipe_new;Left_Swipe_new;0
0,WIN_20180925_17_43_01_Pro_Left_Swipe_new;Left_...
1,WIN_20180925_18_01_40_Pro_Left_Swipe_new;Left_...
2,WIN_20180925_18_03_21_Pro_Left_Swipe_new;Left_...
3,WIN_20180926_16_46_22_Pro_Left_Swipe_new;Left_...
4,WIN_20180926_16_47_09_Pro_Left_Swipe_new;Left_...


In [14]:
train_doc = np.random.permutation(open('/home/datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/home/datasets/Project_data/val.csv').readlines())
batch_size = 16

In [16]:
def generator(source_path, folder_list, batch_size, img_size=(100, 100), num_classes=5):
    print('Source path =', source_path, '; batch size =', batch_size)
    
    img_idx = list(range(30))
    x = len(img_idx)
    y, z = img_size
    
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t) // batch_size
        
        for batch in range(num_batches):
            batch_data = np.zeros((batch_size, x, y, z, 3))
            batch_labels = np.zeros((batch_size, num_classes))

            for folder in range(batch_size):
                folder_name = t[folder + (batch * batch_size)].strip().split(';')[0]
                folder_path = os.path.join(source_path, folder_name)
                imgs = sorted(os.listdir(folder_path))
                
                for idx, item in enumerate(img_idx):
                    img_path = os.path.join(folder_path, imgs[item])
                    image = Image.open(img_path).resize((y, z))
                    image_array = np.array(image).astype(np.float32) / 255.0
                    if image_array.shape[-1] == 3:
                        batch_data[folder, idx, :, :, :] = image_array
                    else:
                        batch_data[folder, idx, :, :, :] = np.stack([image_array]*3, axis=-1)
                
                label = int(t[folder + (batch * batch_size)].strip().split(';')[2])
                batch_labels[folder] = to_categorical(label, num_classes=num_classes)
            
            yield batch_data, batch_labels

        if len(t) % batch_size != 0:
            remaining_size = len(t) % batch_size
            batch_data = np.zeros((remaining_size, x, y, z, 3))
            batch_labels = np.zeros((remaining_size, num_classes))
            
            for folder in range(remaining_size):
                folder_name = t[folder + (num_batches * batch_size)].strip().split(';')[0]
                folder_path = os.path.join(source_path, folder_name)
                imgs = sorted(os.listdir(folder_path))
                
                for idx, item in enumerate(img_idx):
                    img_path = os.path.join(folder_path, imgs[item])
                    image = Image.open(img_path).resize((y, z))
                    image_array = np.array(image).astype(np.float32) / 255.0
                    if image_array.shape[-1] == 3:
                        batch_data[folder, idx, :, :, :] = image_array
                    else:
                        batch_data[folder, idx, :, :, :] = np.stack([image_array]*3, axis=-1)
                
                label = int(t[folder + (num_batches * batch_size)].strip().split(';')[2])
                batch_labels[folder] = to_categorical(label, num_classes=num_classes)
            
            yield batch_data, batch_labels


In [19]:
train_path = '/home/datasets/Project_data/train'
val_path = '/home/datasets/Project_data/val'

num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)

num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)

num_epochs = 20
print('# epochs =', num_epochs)

curr_dt_time = datetime.datetime.now()
print("Current Date and Time:", curr_dt_time)

# training sequences = 663
# validation sequences = 100
# epochs = 20
Current Date and Time: 2024-11-05 19:33:34.802206


In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

def create_conv3d_model(input_shape=(30, 100, 100, 3), num_classes=5):
    model = Sequential()
    model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(BatchNormalization())
    
    model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(BatchNormalization())
    
    model.add(Conv3D(128, kernel_size=(3, 3, 3), activation='relu'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(BatchNormalization())
    
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    
    return model

input_shape = (30, 100, 100, 3)
num_classes = 5
conv3d_model = create_conv3d_model(input_shape=input_shape, num_classes=num_classes)
conv3d_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

2024-11-05 19:34:42.615444: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2024-11-05 19:34:42.615507: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14800 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:3f:00.0, compute capability: 7.5


In [21]:
from tensorflow.keras.optimizers import Adam

optimiser = Adam(learning_rate=0.001)

model = conv3d_model
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 28, 98, 98, 32)    2624      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 14, 49, 49, 32)   0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 14, 49, 49, 32)   128       
 ormalization)                                                   
                                                                 
 conv3d_1 (Conv3D)           (None, 12, 47, 47, 64)    55360     
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 6, 23, 23, 64)    0         
 3D)                                                             
                                                        

In [22]:
batch_size = 16

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [23]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ', '').replace(':', '_') + '/'
if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=1e-6)

callbacks_list = [checkpoint, LR]


In [24]:
if (num_train_sequences % batch_size) == 0:
    steps_per_epoch = int(num_train_sequences / batch_size)
else:
    steps_per_epoch = (num_train_sequences // batch_size) + 1

if (num_val_sequences % batch_size) == 0:
    validation_steps = int(num_val_sequences / batch_size)
else:
    validation_steps = (num_val_sequences // batch_size) + 1

print("Steps per epoch:", steps_per_epoch)
print("Validation steps:", validation_steps)

Steps per epoch: 42
Validation steps: 7


In [25]:
model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=num_epochs,
    verbose=1,
    callbacks=callbacks_list,
    validation_data=val_generator,
    validation_steps=validation_steps,
    class_weight=None,
    workers=1,
    initial_epoch=0
)

Source path = /home/datasets/Project_data/train ; batch size = 16
Epoch 1/20


2024-11-05 19:40:16.000950: I tensorflow/stream_executor/cuda/cuda_dnn.cc:377] Loaded cuDNN version 8302


42/42 [==============================] - ETA: 0s - loss: 3.9513 - categorical_accuracy: 0.5083Source path = /home/datasets/Project_data/val ; batch size = 16

Epoch 00001: saving model to model_init_2024-11-0519_33_34.802206/model-00001-3.95125-0.50830-28.39872-0.18000.h5
42/42 [==============================] - 68s 2s/step - loss: 3.9513 - categorical_accuracy: 0.5083 - val_loss: 28.3987 - val_categorical_accuracy: 0.1800 - lr: 0.0010
Epoch 2/20
42/42 [==============================] - ETA: 0s - loss: 2.2176 - categorical_accuracy: 0.6471
Epoch 00002: saving model to model_init_2024-11-0519_33_34.802206/model-00002-2.21761-0.64706-8.40900-0.18000.h5
42/42 [==============================] - 64s 2s/step - loss: 2.2176 - categorical_accuracy: 0.6471 - val_loss: 8.4090 - val_categorical_accuracy: 0.1800 - lr: 0.0010
Epoch 3/20
42/42 [==============================] - ETA: 0s - loss: 1.2412 - categorical_accuracy: 0.7436
Epoch 00003: saving model to model_init_2024-11-0519_33_34.802206/mod

In [26]:
model.save("final_model.h5")